In [3]:
import ee
import geemap

# Authenticate to Earth Engine
try:
  ee.Initialize(project='ee-ana-zonia')
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')

In [11]:
# region of interest is the entire country
roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/br_biomes").geometry().dissolve()
# select ecoregions to run analysis
ecoregions = ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filterBounds(roi) \
                .map(lambda feature: feature.intersection(roi))

img = ee.Image('projects/ee-ana-zonia/assets/lulc')

## create an empty object to hold stratified points
classes = ee.Image().byte().paint(ecoregions, "class").rename("class")
classes

In [12]:
Map.geemap()
Map.addLayer(classes, {}, 'img')
Map

NameError: name 'Map' is not defined

In [ ]:

## stratify sample points per feature
stratified = classes.addBands(ee.Image.pixelLonLat()).stratifiedSample(
      numPoints = 1000,
      classBand = 'class',
      projection = 'EPSG:4326',
      scale = 30,
      region = ecoregions
)

## add a random number to each feature   
stratified = stratified.randomColumn()

## sample image using stratified points ('random' is the default column name given by randomColumn)
sample = img.sampleRegions(collection=stratified, properties=['class', 'random'], scale=30)


In [40]:
task = ee.batch.Export.table.toDrive(
    collection = fc_sampled,
    description = 'tst_df',
    folder = 'fishnet_tiles'
)

task.start()
